In [1]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from early_stopping import EarlyStopping
import random
from yf_dataset import getInput

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
train_idx, eval_idx, test_idx, weight, yfdata, gnnInputData = getInput(False, False, withMacdSignal=True, macdParamOptimize=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(r'C:\Users\tony\Desktop\GNN_test\result\CrossEntrophyLoss\CrossEntrophyLoss with MACD Signal startLr=1.869129e-04 numNeighbors=24 begin_days=653\best_model.pt')

start_date = 2011-01-30
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
The index for 2013-01-30 is 502


c:\Users\tony\Desktop\GNN_test\yf_dataset.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:97: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:97: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the cave

If there is NAN in data?  False
Index(['2011-01-31 00:00:00 ~ 2011-02-09 00:00:00',
       '2011-02-01 00:00:00 ~ 2011-02-10 00:00:00',
       '2011-02-02 00:00:00 ~ 2011-02-11 00:00:00',
       '2011-02-03 00:00:00 ~ 2011-02-14 00:00:00',
       '2011-02-04 00:00:00 ~ 2011-02-15 00:00:00',
       '2011-02-07 00:00:00 ~ 2011-02-16 00:00:00',
       '2011-02-08 00:00:00 ~ 2011-02-17 00:00:00',
       '2011-02-09 00:00:00 ~ 2011-02-18 00:00:00',
       '2011-02-10 00:00:00 ~ 2011-02-22 00:00:00',
       '2011-02-11 00:00:00 ~ 2011-02-23 00:00:00',
       ...
       '2019-08-08 00:00:00 ~ 2019-08-19 00:00:00',
       '2019-08-09 00:00:00 ~ 2019-08-20 00:00:00',
       '2019-08-12 00:00:00 ~ 2019-08-21 00:00:00',
       '2019-08-13 00:00:00 ~ 2019-08-22 00:00:00',
       '2019-08-14 00:00:00 ~ 2019-08-23 00:00:00',
       '2019-08-15 00:00:00 ~ 2019-08-26 00:00:00',
       '2019-08-16 00:00:00 ~ 2019-08-27 00:00:00',
       '2019-08-19 00:00:00 ~ 2019-08-28 00:00:00',
       '2019-08-20 00

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:211: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



feature_matrix_shape =  torch.Size([2154, 42])
edge size=  torch.Size([2, 34423])
edge_weights size=  torch.Size([34423])
y =  torch.Size([2154, 1])
找到日期 '2013-01-18 00:00:00 ~ 2013-01-30 00:00:00'，index= 495
找到日期 '2017-05-12 00:00:00 ~ 2017-05-23 00:00:00'，index= 1581
找到日期 '2018-03-16 00:00:00 ~ 2018-03-27 00:00:00'，index= 1793
找到日期 '2019-08-20 00:00:00 ~ 2019-08-29 00:00:00'，index= 2152
上漲(0)的数量： 534
持有(1)的数量： 1203
下跌(2)的数量： 417
weight:  [4.033707865093001, 1.7905236907581836, 5.165467625775408]


In [7]:
# mlGAT_period = torch.tensor(np.arange(1361,2671+1),dtype=torch.long)
total_loader = NeighborLoader(gnnInputData, input_nodes=test_idx, num_neighbors=[-1],
                                shuffle=False,
                                num_workers=os.cpu_count() - 2)

In [8]:
model.eval()
model.to(device)
out, var = model.inference(total_loader, device)
# y_pred = out.argmax(dim=-1)
y_pred = torch.exp(out) 

In [2]:
#aggr='weight', corr=0.5
train_idx, eval_idx, test_idx, weight, yfdata, gnnInputData = getInput(False, False, withMacdSignal=True, macdParamOptimize=False, corr=0.6, begin_days=653)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(r'C:\Users\tony\Desktop\GNN_test\result\CrossEntrophyLoss\CrossEntrophyLoss aggr=weight corr=0.6 with MACD Signal startLr=1.869129e-04 numNeighbors=24 begin_days=653\best_model.pt')
# mlGAT_period = torch.tensor(np.arange(1361,2671+1),dtype=torch.long)
total_loader = NeighborLoader(gnnInputData, input_nodes=test_idx, num_neighbors=[-1],
                                shuffle=False,
                                num_workers=os.cpu_count() - 2)
model.eval()
model.to(device)
out, var = model.inference(total_loader, device, gnnInputData)
# y_pred = out.argmax(dim=-1)
y_pred = torch.exp(out) 

start_date = 2009-11-13
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
The index for 2013-01-30 is 806


c:\Users\tony\Desktop\GNN_test\yf_dataset.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yfdata['Signal'].iloc[i] = 0
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yfdata['Signal'].iloc[i] = 2
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yfdata['Signal'].iloc[i] = 0
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:97: SettingWithCopyWar

If there is NAN in data?  False
Index(['2009-11-13 00:00:00 ~ 2009-11-24 00:00:00',
       '2009-11-16 00:00:00 ~ 2009-11-25 00:00:00',
       '2009-11-17 00:00:00 ~ 2009-11-27 00:00:00',
       '2009-11-18 00:00:00 ~ 2009-11-30 00:00:00',
       '2009-11-19 00:00:00 ~ 2009-12-01 00:00:00',
       '2009-11-20 00:00:00 ~ 2009-12-02 00:00:00',
       '2009-11-23 00:00:00 ~ 2009-12-03 00:00:00',
       '2009-11-24 00:00:00 ~ 2009-12-04 00:00:00',
       '2009-11-25 00:00:00 ~ 2009-12-07 00:00:00',
       '2009-11-27 00:00:00 ~ 2009-12-08 00:00:00',
       ...
       '2019-08-08 00:00:00 ~ 2019-08-19 00:00:00',
       '2019-08-09 00:00:00 ~ 2019-08-20 00:00:00',
       '2019-08-12 00:00:00 ~ 2019-08-21 00:00:00',
       '2019-08-13 00:00:00 ~ 2019-08-22 00:00:00',
       '2019-08-14 00:00:00 ~ 2019-08-23 00:00:00',
       '2019-08-15 00:00:00 ~ 2019-08-26 00:00:00',
       '2019-08-16 00:00:00 ~ 2019-08-27 00:00:00',
       '2019-08-19 00:00:00 ~ 2019-08-28 00:00:00',
       '2019-08-20 00

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:204: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  edge_index = torch.tensor([
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:211: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weights = torch.tensor(edge_weights, dtype=torch.float)


feature_matrix_shape =  torch.Size([2458, 42])
edge size=  torch.Size([2, 119646])
edge_weights size=  torch.Size([119646])
y =  torch.Size([2458, 1])
找到日期 '2013-01-18 00:00:00 ~ 2013-01-30 00:00:00'，index= 799
找到日期 '2017-05-12 00:00:00 ~ 2017-05-23 00:00:00'，index= 1885
找到日期 '2018-03-16 00:00:00 ~ 2018-03-27 00:00:00'，index= 2097
找到日期 '2019-08-20 00:00:00 ~ 2019-08-29 00:00:00'，index= 2456
上漲(0)的数量： 618
持有(1)的数量： 1356
下跌(2)的数量： 484
weight:  [3.9773462782527935, 1.8126843657683431, 5.078512396589287]


In [3]:
start_date = "2011-01-30"
end_date = "2019-09-01"
spy = yf.download('SPY', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [9]:
label_data = spy['2018-03-28':]
label_data = label_data[:-1]
init_investment = 10000
investment = init_investment
sAndp_hold = 0
total = []
sell_count = 0
buy_count = 0
for index, pred in enumerate(y_pred):
    open = label_data.iloc[index]['Open']
    adj_close = label_data.iloc[index]['Adj Close']

    if ((pred.argmax(dim=-1) == 0) and (investment >= adj_close)):
        trade_volumn = int (investment / adj_close * pred[pred.argmax(dim=-1)])
        investment -= adj_close * trade_volumn
        sAndp_hold += trade_volumn
        buy_count += trade_volumn
    elif ((pred.argmax(dim=-1) == 2) and (sAndp_hold > 0)):
        trade_volumn = int (investment / adj_close * pred[pred.argmax(dim=-1)])
        investment += adj_close * trade_volumn
        sAndp_hold -= trade_volumn
        sell_count += trade_volumn

    total.append(investment + (sAndp_hold * adj_close))
          
total

[10000.0,
 10000.0,
 10000.0,
 10000.0,
 10000.0,
 10000.0,
 10000.0,
 10000.0,
 10071.374374389648,
 10032.371780395508,
 10093.26254272461,
 10071.374374389648,
 10132.54753112793,
 10212.801467895508,
 10218.416717529297,
 10176.324340820312,
 10112.341430664062,
 10111.221313476562,
 10010.482192993164,
 10028.721939086914,
 10103.925415039062,
 10111.847091674805,
 10046.898742675781,
 10061.788269042969,
 10005.39712524414,
 9987.018600463867,
 10094.738021850586,
 10123.252426147461,
 10123.252426147461,
 10197.985336303711,
 10284.667617797852,
 10313.214584350586,
 10317.687423706055,
 10253.01969909668,
 10292.23503112793,
 10284.32145690918,
 10260.932113647461,
 10331.104202270508,
 10304.960647583008,
 10330.757461547852,
 10310.482803344727,
 10286.947036743164,
 10173.615737915039,
 10303.602920532227,
 10243.134170532227,
 10337.043380737305,
 10384.111862182617,
 10391.354293823242,
 10474.63005065918,
 10473.54606628418,
 10503.236618041992,
 10516.632614135742,
 1052

In [4]:
label_data = spy['2018-03-28':]
label_data = label_data[:-1]
init_investment = 10000
investment = init_investment
sAndp_hold = 0
total = []
sell_count = 0
buy_count = 0
for index, pred in enumerate(y_pred):
    open = label_data.iloc[index]['Open']
    adj_close = label_data.iloc[index]['Adj Close']

    if ((pred.argmax(dim=-1) == 0) and (investment >= adj_close)):
        trade_volumn = 1
        investment -= adj_close * trade_volumn
        sAndp_hold += trade_volumn
        buy_count += trade_volumn
    elif ((pred.argmax(dim=-1) == 2) and (sAndp_hold > 0)):
        trade_volumn = 1
        investment += adj_close * trade_volumn
        sAndp_hold -= trade_volumn
        sell_count += trade_volumn

    total.append(investment + (sAndp_hold * adj_close))
          
total

[10000.0,
 10000.0,
 10000.0,
 10000.0,
 10002.525527954102,
 10004.40835571289,
 9999.049575805664,
 10000.208236694336,
 10007.721328735352,
 10003.946884155273,
 10009.839538574219,
 10007.721328735352,
 10013.641311645508,
 10023.996658325195,
 10024.721206665039,
 10019.289932250977,
 10011.034072875977,
 10010.889541625977,
 9997.89094543457,
 9999.65608215332,
 10006.933837890625,
 10007.839172363281,
 10000.41650390625,
 10002.1181640625,
 9995.673461914062,
 9993.573059082031,
 10005.883850097656,
 10009.957336425781,
 10009.957336425781,
 10021.634353637695,
 10035.321029663086,
 10039.82844543457,
 10040.534683227539,
 10030.323989868164,
 10036.515884399414,
 10035.266372680664,
 10031.573318481445,
 10042.653121948242,
 10037.837203979492,
 10042.589248657227,
 10038.534317016602,
 10033.827163696289,
 10011.160903930664,
 10037.158340454102,
 10025.064590454102,
 10041.92008972168,
 10050.15707397461,
 10051.424499511719,
 10065.997756958008,
 10065.780960083008,
 10071.7

In [10]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=label_data.index, y=list(map(lambda x:x/init_investment, total)), name='Return Percentage'))

fig.update_layout(title="S&P500")
fig.show()
# daily_return = list(map(lambda idx_val: (idx_val[1] - total[idx_val[0] - 1]) / total[idx_val[0] - 1] if idx_val[0] > 0 else 0, enumerate(total)))
print('average_daily_return = ',((total[-1] / init_investment) - 1) / len(total))

average_daily_return =  0.00036386557661391205
